# Lab 9: Grid Localization using Bayes Filter on the real robot

#### <span style="color:rgb(0,150,0)">It is recommended that you close any heavy-duty applications running on your system while working on this lab.</span>

<hr>

In this lab, you are going to you run the grid localization code for your RealRobot in the world created during Lab 7(b).

# Python Classes
There are five major classes (VirtualRobot, Mapper, Localization, Plotter and Trajectory)  defined in **robot_interface.py** that provide you with the necessary functions and variables required for performing grid localization using the Bayes filter. <br>

Refer to lab9_doc.ipynb for documentation.

In [15]:
# Automatically reload changes in python modules
%load_ext autoreload
%autoreload 2

# Import classes
from robot_interface import *
from main import *

import time
import numpy as np
import rospy
from Traj import Trajectory

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [16]:
loop = asyncio.get_event_loop()
asyncio.gather(robotTest(loop))



<_GatheringFuture pending>

In [21]:
theRobot = theRobotHolder.getRobot()
print(theRobot)

In [22]:
await theRobot.ping()

Got pong: round trip 0.06654834747314453
Got pong: round trip 0.06372475624084473
Got pong: round trip 0.062364816665649414
Got pong: round trip 0.07449722290039062
Got pong: round trip 0.07801389694213867
Got pong: round trip 0.06877946853637695
Got pong: round trip 0.12538886070251465
Got pong: round trip 0.06966400146484375
Got pong: round trip 0.054430246353149414
Got pong: round trip 0.08302068710327148
Got pong: round trip 0.059954166412353516
Got pong: round trip 0.057142019271850586
Got pong: round trip 0.04924345016479492
Got pong: round trip 0.07855606079101562
Got pong: round trip 0.04690909385681152
Got pong: round trip 0.05485391616821289
Got pong: round trip 0.054587602615356445
Got pong: round trip 0.054680585861206055
Got pong: round trip 0.05406761169433594
Got pong: round trip 0.05402541160583496
Got pong: round trip 0.07463455200195312
Got pong: round trip 0.05240678787231445
Got pong: round trip 0.05844306945800781
Got pong: round trip 0.053662776947021484
Got pong:

In [23]:
await theRobot.sendMessage("hullo")

Got pong: round trip 0.0781104564666748
Got pong: round trip 0.05009198188781738
Got pong: round trip 0.10205245018005371
BTDebug: Message Received.
Got pong: round trip 0.07053828239440918
Got pong: round trip 0.060787200927734375
Got pong: round trip 0.056662797927856445
Got pong: round trip 0.05490279197692871
Got pong: round trip 0.05002284049987793
Got pong: round trip 0.05509638786315918
Got pong: round trip 0.05834650993347168
Got pong: round trip 0.051064252853393555
Got pong: round trip 0.054459571838378906
Got pong: round trip 0.05870652198791504
Got pong: round trip 0.05150914192199707
Got pong: round trip 0.07352089881896973
Got pong: round trip 0.0712432861328125
Got pong: round trip 0.05591845512390137
Got pong: round trip 0.055544376373291016
Got pong: round trip 0.06870055198669434
Got pong: round trip 0.0544893741607666
Got pong: round trip 0.054976701736450195
Got pong: round trip 0.06637358665466309
Got pong: round trip 0.05453205108642578
Got pong: round trip 0.0562

In [24]:
theRobot = theRobotHolder.getRobot()
myResult = await theRobot.doScan()
print(myResult)

Got pong: round trip 0.07653260231018066
BTDebug: Message Received.
Got pong: round trip 0.07367801666259766
Got pong: round trip 0.07238006591796875
Got pong: round trip 0.055370330810546875
Got pong: round trip 0.05683469772338867
Got pong: round trip 0.05366873741149902
Got pong: round trip 0.058027029037475586
Got pong: round trip 0.05410432815551758
Got pong: round trip 0.04581737518310547
Got pong: round trip 0.05583763122558594
Got pong: round trip 0.07411050796508789
Got pong: round trip 0.07573652267456055
Got pong: round trip 0.06875920295715332
Got pong: round trip 0.055036306381225586
Got pong: round trip 0.07352399826049805
Got pong: round trip 0.055068016052246094
Got pong: round trip 0.05521035194396973
Got pong: round trip 0.05925559997558594
Got pong: round trip 0.07178354263305664
Got pong: round trip 0.05068826675415039
Got pong: round trip 0.07412028312683105
Got pong: round trip 0.05660533905029297
Got pong: round trip 0.05242729187011719
Got pong: round trip 0.075

# Map representation
The map is represented by a set of line segments. Each line segment is described by a start point (x,y) and an end point (x',y').

In the cell below, replace the contents of the 2D array **start_points** such that it contains the start points (x,y) of each line segment. Similarly, replace the contents of the 2D array **end_points** such that it contains the corresponding end points (x',y') of each line segment.

#### Since you are using the world setup during Lab 7(b), you can copy the map points over from Lab7b.ipynb.<br>

In [ ]:
# Start points for each line segment describing the map
start_points = np.array([[0,0],
                         [5,0],
                         [5,5],
                         [0,5],
                         [0,0],
                         [0,1],
                         [1,1],
                         [1,1.5],
                         [0,1.5],
                         [0,3.5],
                         [2,3.5],
                         [2,2.5],
                         [4.25,2.5],
                         [4.25,1.5],
                         [2.5,0.5]])
    
# End points for each line segment describing the map\n",
end_points = np.array([ [5,0],

                       
                       [5,5],
                        [0,5],
                        [0,0],
                        [0,1],
                        [1,1],
                        [1,1.5],
                        [0,1.5],
                        [0,3.5],
                        [2,3.5],
                        [2,2.5],
                        [4.25,2.5],
                        [4.25,1.5],
                        [2.5,0.5],
                        [0,0]])

# Check if map described by start_points and end_points is valid
if(not is_map_valid(start_points, end_points)):
    raise Exception("The definitions of start_points and end_points are not valid. Please make sure the number of points are equal.")

# RealRobot class
You will define the **RealRobot** class that provides the necessary interface to interact with your real robot (get measurement data, get odometry data, motion control) through the Bluetooth module. The **RealRobot** class will be used to replace the **VirtualRobot** class in the Localization package.

This is possible due to the modularity in the localization package, allowing you to "plug and play" different classes. 

#### After defining/modifying the class, re-run the cell below and all the cells that use this class. It is a good idea to test each member function of the class before proceeding to localization.

In [ ]:
class RealRobot(BaseRobot):
    """A class to interact with the real robot
    """

    def __init__(self):
        super().__init__()
        print("Initializing Real Robot")

    def get_pose(self):
        """Get the latest odometry pose data in the map frame.
        
        Do NOT change the arguments or return values of this function.
        
        Returns:
            (x, y, a) (float, float, float): A tuple with latest odometry pose in the map frame 
                                             in the format (x, y, a) with units (meters, meters, degrees)

        """

    async def perform_observation_loop(self, observation_count, rot_vel):
        """ Implement a Bluetooth command, that tells your robot to 
        start an anti-clockwise, rotational scan using PID control on 
        the gyroscope. The scan needs to be a full 360 degree rotation with 
        at least 18 readings from the TOF sensor, with the first reading taken 
        at the current heading of the robot. At the end of the scan, 
        have your robot send back the TOF measurements via Bluetooth. 
        
        If you haven't already, write an automated script to pair down your 
        measurements to 18 approximately equally spaced readings such that 
        the first reading was taken at the heading angle of the robot.
        Use a reasonable rotational speed to achieve this behavior.
        
        Do NOT change the arguments or return values of the function since it will 
        break the localization code. This function is called by the member function 
        "get_obseration_data" in the Localization class (robot_interface.py), 
        with observation_count = 18 and rot_vel = 30. 
        You may choose to ignore the values in the arguments.

        Args:
            observation_count (integer): Number of observations to record
            rot_vel (integer): Rotation speed (in degrees/s)

        Returns:
            obs_range_data (ndarray): 1D array of 'float' type containing observation range data
        """
        arr = await theRobot.doScan()
        return arr
    
    async def set_vel(self, v, w):
        """Set a linear and an angular velocity for your robot.
        
        You will use this function to move the robot.
        It is not used by the Localization class and so you
        may change the arguments and/or return types as needed.

        Args:
            v (integer): Linear velocity
            w (integer): Angular velocity
        """
        await theRobot.move(v, w)
    
    def get_gt_pose(self):
        # Do not change this function
        """Get the latest ground truth pose data
        
        Do NOT change the arguments or return values of this function.

        Returns:
            (x, y, a) (float, float, float): A tuple with latest ground truth pose 
                                             in the format (x, y, a) with units (meters, meters, degress)

        """
        # Since there is no mechanism to find out the ground truth pose of your real robot,
        # it simply returns the odometry pose.
        # This function exists to comply with the design model of the Localization class
        return self.get_pose()

## Instantiate the Mapper class
The **Mapper** class takes in a list of parameters. The names of the arguments should be self explantory.

You may need to change the following parameters based on your map:
- <span style="color:rgb(100,191,100)">min_x</span>: Minimum value of the world coordinates in the x axis
- <span style="color:rgb(100,191,100)">min_y</span>: Minimum value of the world coordinates in the y axis
- <span style="color:rgb(100,191,100)">max_x</span>: Maximum value of the world coordinates in the x axis
- <span style="color:rgb(100,191,100)">max_y</span>: Maximum value of the world coordinates in the y axis
- <span style="color:rgb(100,191,100)">max_cells_x</span>: Total number of grid cells along the x axis. It is given by $(max\_x-min\_x)/cell\_size\_x$.
- <span style="color:rgb(100,191,100)">max_cells_y</span>: Total number of grid cells along the y axis. It is given by $(max\_y-min\_y)/cell\_size\_y$.

The following parameter depends on how your robot is modelled:
- <span style="color:rgb(100,191,100)">obs_per_cell</span>: The number of obersavations made during the rotation behavior. The default is 18.


It is recommended that you do not change the other parameters.

## Instantiate the Localization Class
You will need to set the noise parameters for the odometry motion model and the sensor model. The below cell provides a good starting point for these values. 

Since we utilize Gaussians to model the noise, these noise parameters are essentially the standard deviations of the Gaussians used. If you choose a small standard deivation for a very noise sensor, you are "over-confident" on the values returned by the sensor and hence may localize incorrectly. On the other hand, if you choose a large standard deivation for a precise sensor, then you are are not taking advantage of the precision offered by your sensor and hence may localize poorly (i.e with low confidence). Hence, the noise parameter of the ToF sensor is smaller than that of the odometry (IMU) sensor. The values are in the continous world space and **should be greater than half of the grid cell size**. If not, the noise will not "flow" over to any other grid cell.

In [ ]:
# Instantiate RealRobot to communicate with the real robot

robot = RealRobot()

for i in range(0,2):
    print("Iteration: ", i)
    pose = await bluetooth_get_pose()
    print("Pose: ", pose)
    
    obs = await robot.perform_obsevation_loop()
    print("Pose: ", obs)
    
    print("--------------------------")

# Instantiate Mapper
# Requires a RealRobot object as input
mapper = Mapper(min_x=0, max_x=5, min_y=0, max_y=5, min_a=-180, max_a=180,
                cell_size_x=0.2, cell_size_y=0.2, cell_size_a=20,
                max_cells_x=25, max_cells_y=25, max_cells_a=18,
                ray_length=7, lines=[start_points, end_points], obs_per_cell=18, 
                robot=robot)

# Instantiate Localization
odom_trans_sigma = 0.33
odom_rot_sigma = 15
sensor_sigma = 0.11

# Requires a RealRobot object and a Mapper object as inputs
loc = Localization(robot, mapper, sensor_sigma, odom_trans_sigma, odom_rot_sigma)

# Visualize the map (described using line segments) in the plotter
loc.plotter.visualize_map()

# Peform raycasting and pre-cache the values
mapper.populate_views()

<hr>

# Bayes filter on the real robot
The code cells below provide a skeleton code for various functions required to complete the lab. 

You will need to initialize the belief with a uniform prior ditribution. You may use the member function <span style="color:rgb(100,191,100)">init_uniform_distribution()</span> of class **Localization** to do this. Run the rotation behavior on your robot and run the update step to localize based on the observation data (yes, you skip the prediction step for the first iteration of the Bayes filter).

For subsequent steps, you will need to manually move the robot and run the bayes filter iteration (both prediction and update step) accordingly. Based on your **RealRobot** class and how you control your robot, you will need to modify the functions accordingly. You may also need to modify the functions <span style="color:rgb(100,191,100)">print_prediction_stats()</span> and <span style="color:rgb(100,191,100)">print_update_stats()</span> of class **Localization** in *robot_interface.py*.

In [ ]:
#  Reset the plot, initializes the belief with a uniform distribution, 
# performs the rotation behaviour, and runs the update step
async def init_bayes_filter():
    # Reset Plots
    loc.plotter.reset_plot()

    # Initiize belief
    loc.init_uniform_distribution()

    # Get Observation Data by executing a 360 degree rotation motion
    await loc.get_observation_data()

    # Update Step
    loc.update_step()
    loc.print_update_stats(plot_data=True)

# One iteration of the Bayse filter algorithm
async def step_bayes_filter(current_odom, prev_odom):
    # Prediction Step
    loc.prediction_step(current_odom, prev_odom)
    loc.print_prediction_stats(plot_data=True)

    # Get Observation Data by executing a 360 degree rotation behavior
    await loc.get_observation_data()

    # Update Step
    loc.update_step()
    loc.print_update_stats(plot_data=True)
    

# Records the odom before a robot motion, 
# moves the robot, and records the odom again after motion
async def move_robot():
    prev_odom = robot.get_pose()

    # Code to move your robot goes here
    robot.pose = await robot.move()

    current_odom = robot.get_pose()
    
    return current_odom, prev_odom

In [ ]:
await init_bayes_filter()
current_odom, prev_odom = await move_robot()
await step_bayes_filter(current_odom, prev_odom)

In [ ]:
theRobot.obsData